In [65]:
import pandas as pd
import numpy as np

# leitura dos dados
dados = pd.read_csv("/content/bandaalfaanalise2023semespaco.csv", sep=';', encoding='latin-1')

In [66]:
dados_bandas = pd.read_csv("/content/todasbandas.csv", sep=';', encoding='latin-1')

In [67]:
dados_bandas.head()

,ï»¿usuÃ¡rio,Canal,Valor,Banda,Etapa,InconsistÃªncia,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,72,AF3,"7,372443243",D,6,1,NaN,NaN,NaN
1,72,F7,"16,9324791",D,6,1,NaN,NaN,NaN
2,72,F3,"3,297979026",D,6,1,NaN,NaN,NaN
3,72,FC5,"1,41359911",D,6,1,NaN,NaN,NaN
4,72,T7,"-3,38195214",D,6,1,NaN,NaN,NaN


In [68]:
dados_bandas.rename(columns={"ï»¿usuÃ¡rio": "usuario", "InconsistÃªncia": "inconsistencia"}, inplace=True)

In [69]:
# select only the desired columns and create a new DataFrame
data_band = dados_bandas.loc[:, ['usuario', 'Canal', 'Valor', 'Banda', 'Etapa', 'inconsistencia']]

# print the new DataFrame
print(data_band)

       usuario Canal         Valor Banda  Etapa  inconsistencia
0           72   AF3   7,372443243     D      6               1
1           72    F7    16,9324791     D      6               1
2           72    F3   3,297979026     D      6               1
3           72   FC5    1,41359911     D      6               1
4           72    T7   -3,38195214     D      6               1
...        ...   ...           ...   ...    ...             ...
18055        1    T8  -5,830846015     G      1               0
18056        1   FC6  -6,162481795     G      1               0
18057        1    F4  -2,566195749     G      1               0
18058        1    F8  -7,771323093     G      1               0
18059        1   AF4  -9,827649983     G      1               0

[18060 rows x 6 columns]


In [70]:
dados.head()

,usuario,Canal,Valor,Banda,Etapa,Inconsistencia,QTDCRIT,NATcrit,QTDpergelicit,qtdInconsist
0,72,AF3,"-5,667371975",A,6,1,4,CN,6,1
1,72,F7,"2,725819296",A,6,1,4,CN,6,1
2,72,F3,"-5,568859027",A,6,1,4,CN,6,1
3,72,FC5,"-5,534017306",A,6,1,4,CN,6,1
4,72,T7,"-13,06673307",A,6,1,4,CN,6,1


In [71]:
dados.dtypes

usuario            int64
Canal             object
Valor             object
Banda             object
Etapa              int64
Inconsistencia     int64
QTDCRIT            int64
NATcrit           object
QTDpergelicit      int64
qtdInconsist       int64
dtype: object

Filter only the frontal channels: 'AF3', 'AF4', 'F3', 'F4', 'FC5', 'FC6'.

In [72]:
# list of allowed channels
allowed_channels = ['AF3', 'AF4', 'F3', 'F4', 'FC5', 'FC6']

# filter the dataset
df_filtered = data_band[data_band['Canal'].isin(allowed_channels)]

# print the filtered dataset
print(df_filtered.head(30))

    usuario Canal         Valor Banda  Etapa  inconsistencia
0        72   AF3   7,372443243     D      6               1
2        72    F3   3,297979026     D      6               1
3        72   FC5    1,41359911     D      6               1
10       72   FC6   5,878939895     D      6               1
11       72    F4   4,648282591     D      6               1
13       72   AF4   1,530374833     D      6               1
14       72   AF3  -5,597850078     T      6               1
16       72    F3  -5,289902074     T      6               1
17       72   FC5  -5,090933159     T      6               1
24       72   FC6  -2,746000926     T      6               1
25       72    F4  -3,481781932     T      6               1
27       72   AF4  -8,895644802     T      6               1
28       72   AF3  -5,667371975     A      6               1
30       72    F3  -5,568859027     A      6               1
31       72   FC5  -5,534017306     A      6               1
38       72   FC6  -3,67

Here, if the p-value is less than or equal to 0.05, the value stored in "p-value_0.05" will be True, otherwise it will be False. Similarly, if the p-value is less than or equal to 0.10, the value stored in "p-value_0.10" will be True, otherwise it will be False.

In [73]:

from scipy.stats import ttest_ind


# Convert the "Valor" column from string to float
data_band["Valor"] = data_band["Valor"].str.replace(",", ".").astype(float)

# Create a list of the "Banda" categories
band_categories = ["D", "B", "A", "T", "G"]

# Create an empty dataframe to store the results
results_df = pd.DataFrame(columns=["Banda", "Etapa", "inconsistencia_0_mean", "inconsistencia_1_mean", "p-value_0.05", "p-value_0.10"])

# Loop through each category of "Banda"
for banda in band_categories:
  # Loop through each "Etapa" from 1 to 4
  for etapa in range(1, 5):
    # Get the samples where "inconsistencia" equals 0 and 1 for the current "Banda" and "Etapa"
    inconsistencia_0_sample = data_band[(data_band["inconsistencia"] == 0) & (data_band["Banda"] == banda) & (data_band["Etapa"] == etapa)]["Valor"]
    inconsistencia_1_sample = data_band[(data_band["inconsistencia"] == 1) & (data_band["Banda"] == banda) & (data_band["Etapa"] == etapa)]["Valor"]
    # Perform the independent samples t-test and get the p-value
    t, p = ttest_ind(inconsistencia_0_sample, inconsistencia_1_sample, equal_var=False)
    # Calculate the means of the samples
    inconsistencia_0_mean = inconsistencia_0_sample.mean()
    inconsistencia_1_mean = inconsistencia_1_sample.mean()
    # Append the results to the dataframe
    results_df = results_df.append({"Banda": banda, "Etapa": etapa, "inconsistencia_0_mean": inconsistencia_0_mean, "inconsistencia_1_mean": inconsistencia_1_mean, "p-value_0.05": p <= 0.05, "p-value_0.10": p <= 0.10}, ignore_index=True)

# Print the results
print(results_df)


<ipython-input-73-b36e8afee72d>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"Banda": banda, "Etapa": etapa, "inconsistencia_0_mean": inconsistencia_0_mean, "inconsistencia_1_mean": inconsistencia_1_mean, "p-value_0.05": p <= 0.05, "p-value_0.10": p <= 0.10}, ignore_index=True)
<ipython-input-73-b36e8afee72d>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({"Banda": banda, "Etapa": etapa, "inconsistencia_0_mean": inconsistencia_0_mean, "inconsistencia_1_mean": inconsistencia_1_mean, "p-value_0.05": p <= 0.05, "p-value_0.10": p <= 0.10}, ignore_index=True)
<ipython-input-73-b36e8afee72d>:26: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to b

   Banda Etapa  inconsistencia_0_mean  inconsistencia_1_mean p-value_0.05  \
0      D     1              11.590727              12.247705        False   
1      D     2              15.613377              16.192707        False   
2      D     3              14.777220              15.817433         True   
3      D     4              13.460935              14.563249        False   
4      B     1              -7.657287              -6.658988         True   
5      B     2              -2.997693              -0.922467         True   
6      B     3              -3.914016              -2.351160         True   
7      B     4              -5.695633              -4.783557        False   
8      A     1              -4.470849              -3.897638        False   
9      A     2              -0.110820               1.127637         True   
10     A     3              -0.989808               0.072586         True   
11     A     4              -2.822545              -1.895531        False   

<ipython-input-73-b36e8afee72d>:26: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  results_df = results_df.append({"Banda": banda, "Etapa": etapa, "inconsistencia_0_mean": inconsistencia_0_mean, "inconsistencia_1_mean": inconsistencia_1_mean, "p-value_0.05": p <= 0.05, "p-value_0.10": p <= 0.10}, ignore_index=True)


In [74]:
results_df.head(20)

,Banda,Etapa,inconsistencia_0_mean,inconsistencia_1_mean,p-value_0.05,p-value_0.10
0,D,1,11.590727,12.247705,False,False
1,D,2,15.613377,16.192707,False,False
2,D,3,14.777220,15.817433,True,True
3,D,4,13.460935,14.563249,False,True
4,B,1,-7.657287,-6.658988,True,True
5,B,2,-2.997693,-0.922467,True,True
6,B,3,-3.914016,-2.351160,True,True
7,B,4,-5.695633,-4.783557,False,True
8,A,1,-4.470849,-3.897638,False,False
9,A,2,-0.110820,1.127637,True,True
